In [1]:
import pandas
import tarfile
import glob
import json
import os
import re
import numpy as np
from functools import reduce
from IPython.core.display import display, HTML

Let's ingest a bunch of data!

In [2]:
PATH_RE = re.compile(r"(.*?/)*out-([a-zA-Z0-9_-]+)-(ice|mcmc|z3|freqhorn)--(.*?)--i\d+\.tar\.gz$")

In [3]:
FREQHORN_V2_BOOT_RE = re.compile(r'\s*Success after (.+?\s+)?bootstrapping\s*')
FREQHORN_V2_SAMPLING_RE = re.compile(r'\s*Success after (.+?\s+)?sampling\s*')

def jsons_from_targlob(tarpath):
    for path in glob.glob(os.path.expanduser(tarpath)):
        path_match = PATH_RE.match(path)
        with tarfile.open(path) as tf:
            results, manual_success_kind = None, None
            for tarinfo in tf:
                if tarinfo.name.lower() == 'out/out.log':
                    outlog = tf.extractfile(tarinfo)
                    out_log_lines = outlog.read().decode('utf-8').splitlines()
                    if any(FREQHORN_V2_BOOT_RE.match(line) for line in out_log_lines):
                        manual_success_kind = 'bootstrapping'
                    elif any(FREQHORN_V2_SAMPLING_RE.match(line) for line in out_log_lines):
                        manual_success_kind = 'sampling'
                elif tarinfo.name.lower() == 'out/result.json':
                    results = json.load(tf.extractfile(tarinfo))
                    if not results['hyperparams'] and path_match:
                        # workaround: some early runs didn't save hyperparams
                        results['hyperparams'] = path_match.group(4)
            if results is not None:
                if manual_success_kind:
                    results['successKind'] = manual_success_kind
                yield results
SRC_GLOBS = ["~/Desktop/runs/linux0/*.tar.gz",
             "~/Desktop/runs/linux1/*.tar.gz",
             "~/Desktop/runs/linux2/*.tar.gz",
             "~/Desktop/runs/windows0/*.tar.gz",
             "~/Desktop/runs/windows1/*.tar.gz",
             "~/Desktop/runs/windows2/*.tar.gz",
             "~/Desktop/runs/windows3/*.tar.gz",
             "/Users/skaufman/Code/grigory-aeval/tools/deep_bench/windows/*.tar.gz"]
all_tar_paths = reduce(lambda a, b: a + glob.glob(os.path.expanduser(b)), SRC_GLOBS, [])

In [4]:
src = reduce(lambda a, b: a + list(jsons_from_targlob(b)), SRC_GLOBS, [])
df = pandas.DataFrame(src)
df.drop(columns=['startDate'], inplace=True)  # startDate is boring

Let's add an extra column--`bestTime`--taking `reportedTime` where available and `processTime` otherwise. We use `min` because `processTime` must, of course, always be smaller. Then drop the others. Who needs 'em.

In [5]:
df['bestTime'] = df[['processTime', 'reportedTime']].min(axis=1)
df.drop(columns=['processTime', 'reportedTime'], inplace=True)

#### Flagging Failures
Failures have no `results.json` files. In general, I'm not immediately sure about why failures occur. In the case of **ICE**, I think syntactic details matter; if I had more time, I'd try rewriting a lot of these such that they resemble ICE's original PLDI benchmarks. **MCMC** seems to segfault arbitrarily. In any case, let's flag all MCMC & ICE trials with missing results.json files `outcome = "failure"`.

At the same time, a few of our tarballs might've resulted from non-existant benchmarks. Let's skip those/not include them in any subsequent analysis.

In [6]:
def line_is_nonexist(line):
    pat = r"^\s*OSError: \[Errno \d+\] No such [\w\s]+: '([\d\w\s\/]+\/)?([\w\d_\-]+)'\s*$"
    m = re.match(pat, line)
    if m:
        return True
    return False

In [7]:
path_is_ice = lambda pth: "-ice-" in pth.lower()
path_is_mcmc = lambda pth: "-mcmc-" in pth.lower()
path_is_spacer = lambda pth: "-z3-" in pth.lower()
tarinfo_is_result = lambda tarinfo: tarinfo.name.lower() == 'out/result.json'
tarinfo_is_supervisor_log = lambda tarinfo: tarinfo.name.lower() == 'out/supervisor.std.log'
        
for path in all_tar_paths:
    if path_is_ice(path):
        algo = "ICE"
    elif path_is_mcmc(path):
        algo = "MCMC"
    elif path_is_spacer(path):
        also = "Z3"
    else:
        continue
    has_results, is_nonexist = False, False
    with tarfile.open(path) as tf:
        for tarinfo in tf:
            if tarinfo_is_result(tarinfo):
                has_results = True
            elif tarinfo_is_supervisor_log(tarinfo):
                superlog = tf.extractfile(tarinfo)
                if any(line_is_nonexist(l) for l in superlog.read().decode('utf-8').splitlines()):
                    is_nonexist = True
    if not has_results and not is_nonexist:
        name = re.match(r'out-(.+?)-(ice|mcmc|z3)-.*\.tar\.gz', os.path.split(path)[1]).group(1)
        df = df.append({'algorithm': algo, 'benchmarkName': name, 'outcome': 'failure'}, ignore_index=True)

#### Any FreqHorn  failures?
Before we go further, let's look for failures (as opposed to timeouts) from FreqHorn. These should have no `results.json` files. (Also, ignore the Spacer, ICE, and MCMC failures, which we used prior.)

In [8]:
failure_paths = []
for path in all_tar_paths:
    if any(fn(path) for fn in [path_is_ice, path_is_mcmc, path_is_spacer]):
        continue
    found_results = False
    with tarfile.open(path) as tf:
        for tarinfo in tf:
            if tarinfo_is_result(tarinfo):
                found_results = True
                break
    if not found_results:
        failure_paths.append(path)
if failure_paths:
    display(HTML("<strong>Missing results.json in:</strong><ul>"))
    for pth in sorted(failure_paths):
        display(HTML("<li>%s</li>" % str(os.path.split(os.path.splitext(os.path.splitext(pth)[0])[0])[1])))
    display(HTML("</ul>"))
else:
    display(HTML("<strong>No failures.</strong>"))

In [9]:
# Where are out ICE abdu_04
df[df['benchmarkName'] == 'abdu_04']

,algorithm,benchmarkName,hyperparams,iters,outcome,successKind,bestTime
32,FREQHORN,abdu_04,v1-eps-freqs-aggp,NaN,success,NaN,11.104865
304,FREQHORN,abdu_04,v1-eps-freqs,NaN,success,NaN,41.610121
439,FREQHORN,abdu_04,v1-eps-freqs,NaN,success,NaN,28.415789
453,FREQHORN,abdu_04,v1,NaN,success,NaN,2.544792
606,FREQHORN,abdu_04,freqs-aggp,NaN,success,bootstrapping,0.490420
675,FREQHORN,abdu_04,aggp-itp3,NaN,success,bootstrapping,0.533299
883,FREQHORN,abdu_04,v1-eps,NaN,timeout,NaN,NaN
1148,FREQHORN,abdu_04,aggp,NaN,success,bootstrapping,0.484320
1366,FREQHORN,abdu_04,aggp,NaN,success,bootstrapping,0.489147
1449,FREQHORN,abdu_04,v1-eps,NaN,timeout,NaN,NaN


#### Benchmark Success
Which benchmarks are available to each algorithm and didn't uniformly fail?

In [10]:
with pandas.option_context('display.max_rows', None):
    display(df[df['outcome'] != 'failure'][['benchmarkName', 'algorithm']].pivot_table(index='benchmarkName', columns='algorithm', aggfunc=lambda x: 'Y'))

algorithm,FREQHORN,ICE,MCMC,Z3
benchmarkName,,,,
abdu_01,Y,Y,Y,Y
abdu_02,Y,Y,Y,Y
abdu_03,Y,Y,Y,Y
abdu_04,Y,None,Y,Y
bhmr2007_true-unreach-call,Y,None,Y,Y
bouncy_three_counters_merged,Y,None,Y,Y
bouncy_two_counters_merged,Y,Y,Y,Y
cegar1,Y,None,Y,Y
cegar2,Y,None,Y,Y


#### Timeout Counts

In [11]:
df.groupby(['algorithm', 'hyperparams', 'outcome']).size()

algorithm  hyperparams        outcome     
FREQHORN   aggp               success         434
                              timeout          82
           aggp-itp3          success         431
                              timeout          84
           freqs-aggp         success         433
                              timeout          82
           freqs-aggp-itp3    success         436
                              timeout          80
           v1                 iterLimitHit      9
                              success         306
                              timeout         201
           v1-eps             iterLimitHit      9
                              success         310
                              timeout         197
           v1-eps-freqs       iterLimitHit      9
                              success         332
                              timeout         175
           v1-eps-freqs-aggp  iterLimitHit     43
                              success         373
       

In [12]:
df.groupby(['algorithm', 'hyperparams']).size()

algorithm  hyperparams      
FREQHORN   aggp                 516
           aggp-itp3            515
           freqs-aggp           515
           freqs-aggp-itp3      516
           v1                   516
           v1-eps               516
           v1-eps-freqs         516
           v1-eps-freqs-aggp    516
ICE                             215
MCMC                            314
Z3         spacerhyp1           495
           spacerhyp2           516
dtype: int64

## Main Analysis

What are the mean variations for each algorithm? We expect them to be very slow for ICE and Z3/Spacer and much higher for probabilistic methods like MCMC and FreqHorn.

In [13]:
time_vars = df.groupby(['algorithm', 'benchmarkName'])['bestTime'].var().rename("timeVariation")
time_vars = time_vars[time_vars.isnull() == False]
time_vars = time_vars.reset_index()
time_vars = time_vars.pivot(index="benchmarkName", columns="algorithm")
time_vars['timeVariation'].mean()

algorithm
FREQHORN     91.174769
ICE           0.006892
MCMC        102.484544
Z3            3.793790
dtype: float64

#### Cross-Algorithm Time Summary
First, figure out how many benchmarks (with any _outcome_) were run for each algorithm. We'll use this in the next step.

In [14]:
holes = df.pivot_table(index='benchmarkName',
                       columns=['algorithm'],
                       values='bestTime',
                       dropna=False,
                       aggfunc=lambda x: x.size)
totally_ported = holes.isnull().any(axis=1) == False
print("Not totally ported:")
totally_ported[totally_ported == False]

Not totally ported:


benchmarkName
const_div_3                  False
const_div_4                  False
const_mod_3                  False
css2003_true-unreach-call    False
dillig02                     False
dillig08                     False
dillig10                     False
dillig18                     False
dillig19                     False
dillig20-1                   False
dillig20-2                   False
dillig20-3                   False
dillig21                     False
dillig22                     False
dillig22-1                   False
dillig22-2                   False
dillig22-3                   False
dillig22-4                   False
dillig22-5                   False
dillig22-6                   False
dillig42                     False
dillig42-1                   False
dillig44                     False
dillig44-1                   False
dillig46                     False
fig3                         False
gcd_1                        False
gcd_2                        False
gcd_3 

We use **min** for non-probabilistic methods (*ICE* and *Z3*) and **mean** for others. Only include benchmarks which we've ported to all algorithms.

In [15]:
for x in df[['algorithm', 'hyperparams']]:
    print(x)

algorithm
hyperparams


In [16]:
time_summary_parts = {}
for algo in df['algorithm'].unique():
    u_hypers = ['']
    if algo.upper() not in ('ICE', 'MCMC'):
        u_hypers = df[df['algorithm'] == algo]['hyperparams'].unique()
    for hyper in u_hypers:
        per_bench = df[df['algorithm'] == algo][df['hyperparams'] == hyper].groupby('benchmarkName')['bestTime']
        if algo.upper() in ('ICE', 'Z3'):
            col_name = algo.lower() + "-" + hyper + "MinTime"
            per_bench = per_bench.min()
        else:
            col_name = algo.lower() + "-" + hyper + "AvgTime"
            per_bench = per_bench.mean()
        time_summary_parts[col_name] = per_bench
final_time_summary = pandas.concat(time_summary_parts, axis=1)[totally_ported]
final_time_summary.to_csv("final_time_summary.csv")
final_time_summary

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


,freqhorn-aggp-itp3AvgTime,freqhorn-aggpAvgTime,freqhorn-freqs-aggp-itp3AvgTime,freqhorn-freqs-aggpAvgTime,freqhorn-v1-eps-freqs-aggpAvgTime,freqhorn-v1-eps-freqsAvgTime,freqhorn-v1-epsAvgTime,freqhorn-v1AvgTime,ice-MinTime,mcmc-AvgTime,z3-spacerhyp1MinTime,z3-spacerhyp2MinTime
abdu_01,0.361633,0.322749,0.362862,0.326144,2.749041,5.032037,38.832157,2.382093,NaN,NaN,0.038595,0.038058
abdu_02,0.586453,36.345643,0.593720,53.077623,40.884721,34.932443,32.175781,13.856761,NaN,NaN,0.047519,0.052318
abdu_03,0.705050,NaN,0.704973,NaN,49.529664,NaN,NaN,30.962278,NaN,NaN,0.053647,0.051166
abdu_04,0.535660,0.485685,0.539468,0.489814,10.608264,25.641194,NaN,2.745355,NaN,NaN,0.047029,0.042643
bhmr2007_true-unreach-call,0.746312,NaN,0.745017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.047970,0.047481
bouncy_three_counters_merged,30.705833,38.357950,NaN,55.940787,49.066166,NaN,19.226857,NaN,NaN,30.601000,NaN,NaN
bouncy_two_counters_merged,6.343885,6.258905,7.671132,4.089318,3.492176,5.424123,7.093249,13.805033,NaN,NaN,NaN,NaN
cegar1,0.307645,0.277290,0.309956,0.276082,3.785301,18.345265,4.518865,8.432661,NaN,3.813000,0.039644,0.051217
cegar2,0.901051,0.905326,1.325988,0.860794,2.162391,1.831107,1.048009,1.853755,NaN,0.301000,0.061322,0.046168
cggmp_iter_1,1.123942,2.004784,6.618948,5.032795,14.534831,NaN,9.181728,NaN,0.33,18.353000,0.054347,0.051785


What are the counts after bootstrapping and sampling?

In [61]:
freqhorn_hypers = df[df['algorithm'] == 'FREQHORN']['hyperparams'].unique()
for hyper in freqhorn_hypers:
    print(hyper)
    for kind in ('bootstrapping', 'sampling'):
        aa = df[df['algorithm'] == 'FREQHORN'][df['hyperparams'] == hyper][df['outcome'] == 'success'][df['successKind'] == kind] \
            .groupby(['benchmarkName']).count()[totally_ported]['algorithm'].sum()
        print(str(kind) + ": " + str(aa))
    print("")
# aa[df['successKind'].isnull() == False].groupby(['successKind'])['bestTime'].count()

aggp
bootstrapping: 204
sampling: 87

freqs-aggp
bootstrapping: 204
sampling: 78

v1-eps-freqs
bootstrapping: 0
sampling: 0

v1-eps-freqs-aggp
bootstrapping: 0
sampling: 0

v1-eps
bootstrapping: 0
sampling: 0

v1
bootstrapping: 0
sampling: 0

aggp-itp3
bootstrapping: 222
sampling: 67

freqs-aggp-itp3
bootstrapping: 222
sampling: 67



/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [20]:
df[df['algorithm'] == 'FREQHORN'][df['successKind'].isnull() == False].groupby(['hyperparams', 'successKind'])['bestTime'].count()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


hyperparams      successKind  
aggp             bootstrapping    309
                 sampling         125
aggp-itp3        bootstrapping    351
                 sampling          80
freqs-aggp       bootstrapping    309
                 sampling         124
freqs-aggp-itp3  bootstrapping    351
                 sampling          85
Name: bestTime, dtype: int64